In [1]:
# data imports
import pandas as pd
import requests
import json

# plotting imports
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
import bqplot as bq
import ipywidgets as widgets
from IPython.display import display

In [2]:
populations = pd.read_csv('https://raw.githubusercontent.com/dsoto/simple-covid-models/master/state-populations.csv',
                          index_col='USPS')
populations.head()

,Name,FIPS,population
USPS,,,
AL,Alabama,1,4903185
AK,Alaska,2,731545
AZ,Arizona,4,7278717
AR,Arkansas,5,3017804
CA,California,6,39512223


In [3]:
URL = 'https://covidtracking.com/api/states'
r = requests.get(URL)
covid = pd.read_json(json.dumps(r.json()))
covid = covid.set_index('state')
covid = covid[['positive', 'death', 'totalTestResults']]
covid['population'] = populations['population']
covid['FIPS'] = populations['FIPS']

covid['death_per_capita']    = covid['death'] / covid['population']
covid['tests_per_capita']    = covid['totalTestResults'] / covid['population']
covid['case_fatality_ratio'] = covid['death'] / covid['positive']
covid['positive_testing_ratio']      = covid['positive'] / covid['totalTestResults']
covid['positive_per_capita'] = covid['positive'] / covid['population']

covid = covid.dropna()
covid['FIPS'] = covid['FIPS'].astype('int').astype('str')
covid.head()

,positive,death,totalTestResults,population,FIPS,death_per_capita,tests_per_capita,case_fatality_ratio,positive_testing_ratio,positive_per_capita
state,,,,,,,,,,
AK,85,2.0,2921,731545.0,2,2.733940e-06,0.003993,0.023529,0.029100,0.000116
AL,644,3.0,4828,4903185.0,1,6.118472e-07,0.000985,0.004658,0.133389,0.000131
AR,388,3.0,3310,3017804.0,5,9.941003e-07,0.001097,0.007732,0.117221,0.000129
AZ,738,13.0,8193,7278717.0,4,1.786029e-06,0.001126,0.017615,0.090077,0.000101
CA,3879,78.0,21259,39512223.0,6,1.974073e-06,0.000538,0.020108,0.182464,0.000098


In [4]:
def update_plot(*args):
    color_data = dict(zip(covid['FIPS'], covid[dropdown.value]))
        
    sc_cl = bqs.ColorScale(scheme='viridis')
    axis = bqa.ColorAxis(scale=sc_cl, num_ticks=5)
    map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
    states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)
    fig.marks = [states_map]
    fig.axes = [axis]
    
dropdown = widgets.Dropdown(options=['positive', 'positive_per_capita', 
                                     'death', 'death_per_capita', 
                                     'totalTestResults', 'tests_per_capita', 
                                     'case_fatality_ratio', 'positive_testing_ratio'])
dropdown.observe(update_plot, 'value')

color_data = dict(zip(covid['FIPS'], covid[dropdown.value])) 
sc_cl = bqs.ColorScale(scheme='viridis')
sc_geo = bqs.AlbersUSA(scale_factor=1200, translate=(500,400))
axis = bqa.ColorAxis(scale=sc_cl, num_ticks=5)
map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)

fig = bq.Figure(marks=[states_map], axes=[axis], title='COVID Metrics By State')
box = widgets.VBox([fig, dropdown])
display(box)